# Workflow

## Collect Parameters and Providers
### Simulation(McStas) Data
There is a dedicated loader, ``load_mcstas_nmx_file`` for ``McStas`` simulation data workflow. <br>
``MaximumProabability`` can be manually provided to the loader derive more realistic number of events. <br>
It is because ``weights`` are given as probability, not integer number of events in a McStas file. <br>

In [ ]:
# Collect parameters and providers
import scipp as sc
from ess.nmx.workflow import collect_default_parameters
from ess.nmx.mcstas_loader import load_mcstas_nmx_file
from ess.nmx.mcstas_loader import (
    InputFilename,
    MaximumProbability,
    DefaultMaximumProbability
)
from ess.nmx.data import small_mcstas_sample

providers = (load_mcstas_nmx_file, )

file_path = small_mcstas_sample()  # Replace it with your data file path
params = {
    **collect_default_parameters(),
    InputFilename: InputFilename(file_path),
    # Additional parameters for McStas data handling.
    MaximumProbability: DefaultMaximumProbability,
}

In [ ]:
from typing import get_type_hints
param_reprs = {key.__name__: value for key, value in params.items()}
prov_reprs = {get_type_hints(prov)['return'].__name__: prov.__name__ for prov in providers}

# Providers and parameters to be used for pipeline
sc.DataGroup(**prov_reprs, **param_reprs)

## Build Workflow

In [ ]:
import sciline as sl
from ess.nmx.mcstas_loader import NMXData

nmx_pl = sl.Pipeline(list(providers), params=params)
nmx_workflow = nmx_pl.get(NMXData)
nmx_workflow.visualize()

## Compute Desired Types

In [ ]:
# Event data grouped by detector panel and pixel id.
da = nmx_workflow.compute(NMXData)
da